# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_cities_df = pd.read_csv('../Output_data/cities.csv')

del vacation_cities_df['City_ID']

vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Keti Bandar,24.14,67.45,87.82,61,0,10.89,PK,1603113615
1,Kieta,-6.22,155.63,79.90,76,76,2.35,PG,1603113615
2,Saint Paul Harbor,57.79,-152.41,40.19,72,0,5.79,US,1603113616
3,Jamestown,42.10,-79.24,50.00,93,100,3.00,US,1603113616
4,Shenjiamen,29.96,122.30,66.04,71,25,15.41,CN,1603113616
...,...,...,...,...,...,...,...,...,...
578,Zapolyarnyy,69.42,30.81,32.79,94,92,7.49,RU,1603113756
579,Nouadhibou,20.93,-17.03,81.25,52,0,8.52,MR,1603113756
580,Hervey Bay,-25.30,152.85,70.00,90,51,2.35,AU,1603113756
581,Sheltozero,61.37,35.37,39.36,77,25,13.00,RU,1603113756


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)


In [7]:
# Store latitude and longitude in locations
locations = vacation_cities_df[["Lat", "Lng"]]
weights = vacation_cities_df["Humidity"].astype(float)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df.dtypes


preferred_cities_df = vacation_cities_df.loc[(vacation_cities_df['Max Temp'] >= 70) & (vacation_cities_df['Max Temp'] <= 80) 
                          & (vacation_cities_df['Humidity'] <= 85) & (vacation_cities_df['Humidity'] >= 15)
                           & (vacation_cities_df['Wind Speed'] <= 15) & (vacation_cities_df['Cloudiness'] <= 25)
                          ]
preferred_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Mahébourg,-20.41,57.70,78.01,62,0,10.00,MU,1603113617
38,Kapurthala Town,31.38,75.38,79.63,16,0,5.84,IN,1603113623
40,Hong Kong,22.29,114.16,78.01,66,0,1.01,HK,1603113353
55,Kapaa,22.08,-159.32,73.00,59,5,9.33,US,1603113627
74,Honiara,-9.43,159.95,78.78,78,7,5.35,SB,1603113420


In [18]:
preferred_cities_df['Hotel Name'] = ""

preferred_cities_df = preferred_cities_df.reset_index(drop=True)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
params = {'key': g_key,
             'type': 'lodging',
              'radius': 5000
             }

def findnearby(params):
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=params)
    data = response.json()
    return data
    
for index, row in preferred_cities_df.iterrows():
    city = row["City"]
    print(f'Looking for hotels in index no:{index} | {city}')
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    hotel_data = findnearby(params)
    
    try:
        preferred_cities_df.loc[index,'Hotel Name'] = hotel_data['results'][0]['name']

    except:
        print(f"missing value for row number{index}")
    
    


Looking for hotels in index no:0 | Mahébourg
Looking for hotels in index no:1 | Kapurthala Town
Looking for hotels in index no:2 | Hong Kong
Looking for hotels in index no:3 | Kapaa
Looking for hotels in index no:4 | Honiara
Looking for hotels in index no:5 | Castro
Looking for hotels in index no:6 | Avarua
Looking for hotels in index no:7 | Cidreira
Looking for hotels in index no:8 | Mossamedes
Looking for hotels in index no:9 | Mananjary
Looking for hotels in index no:10 | Cabo San Lucas
Looking for hotels in index no:11 | Beyneu
Looking for hotels in index no:12 | Puerto Escondido
Looking for hotels in index no:13 | Cap Malheureux
Looking for hotels in index no:14 | Souillac
Looking for hotels in index no:15 | Inhambane
Looking for hotels in index no:16 | Saint-Pierre
Looking for hotels in index no:17 | Quatre Cocos
Looking for hotels in index no:18 | Bambous Virieux
Looking for hotels in index no:19 | Koumac
Looking for hotels in index no:20 | Choa Saidan Shah
Looking for hotels in

In [20]:
preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mahébourg,-20.41,57.70,78.01,62,0,10.00,MU,1603113617,Shandrani Beachcomber Resort & Spa
1,Kapurthala Town,31.38,75.38,79.63,16,0,5.84,IN,1603113623,Hotel Mehak
2,Hong Kong,22.29,114.16,78.01,66,0,1.01,HK,1603113353,Mini Hotel Central
3,Kapaa,22.08,-159.32,73.00,59,5,9.33,US,1603113627,Sheraton Kauai Resort at Coconut Beach
4,Honiara,-9.43,159.95,78.78,78,7,5.35,SB,1603113420,Heritage Park Hotel
5,Castro,-24.79,-50.01,72.23,66,2,8.23,BR,1603113634,CHACARA BAILLY
6,Avarua,-21.21,-159.78,75.33,77,4,5.28,CK,1603113411,Paradise Inn
7,Cidreira,-30.18,-50.21,72.39,65,0,11.97,BR,1603113643,Hotel Castelo
8,Mossamedes,-15.20,12.15,73.29,68,23,10.76,AO,1603113645,Hotel Chik Chik Namibe
9,Mananjary,-21.22,48.33,76.55,69,11,8.23,MG,1603113648,Vahiny Lodge


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in preferred_cities_df.iterrows()]
marker_locations = preferred_cities_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map


locations = vacation_cities_df[["Lat", "Lng"]]
weights = vacation_cities_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# markers = gmaps.marker_layer(locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])

markers = gmaps.marker_layer(marker_locations,
    info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))